In [99]:
import pickle
import re
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

### Order to run these:
##### To get games up to this point:
db = wnba_update(2019) <br>
db = db_format(db)

### To get today's scheduled games: 
today_games = todays_games(date)

### Predictions:
predict_db(db, today_games)

In [100]:
def wnba_scrape(team, year):

    # Date format = YYYYMMDD
    # 1. Create schedule from basketball-reference
    # 2. Get basic stats for just the teams
    # 3. Match opponent stats using indexes (Different function)
    # 4. Comupte rolling averages (different function?)
    # 5. Get betting lines (different function)

    def date_formatter(date):

        if len(str(date.month)) != 2:
            month = '0'+ str(date.month)
        else:
            month = str(date.month)

        if len(str(date.day)) != 2:
            day = '0' + str(date.day)
        else:
            day = str(date.day)

        newdate = str(date.year) + month + day
        return newdate

    team_abbrevs = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
    }

    url = f'https://www.basketball-reference.com/wnba/schedules/{team}/{year}.html'
    res = requests.get(url, 'html.parser')
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    reg_season = soup.find('table',{'id':'teams_games'})

    oppts = reg_season.find_all(attrs = {'data-stat':'opp_name'})
    opps = [entry.get_text() for entry in oppts if entry.get_text() != "Opponent"]
    opp_abbrev = [team_abbrevs[opp] for opp in opps]

    datesoup = reg_season.find_all(attrs = {'data-stat':'date_game'})
    dates = [entry.get_text() for entry in datesoup if entry.get_text() != 'Date']

    souppoints = reg_season.find_all(attrs = {'data-stat':'pts'})
    soupopp = reg_season.find_all(attrs = {'data-stat':'opp_pts'})

    teampoints = [entry.get_text() for entry in souppoints if not entry.get_text().startswith('T')]
    opppoints = [entry.get_text() for entry in soupopp if not entry.get_text().startswith('O')]

    teampoints = list(map(lambda x: int(x), teampoints))
    opppoints = list(map(lambda x: int(x), opppoints))

    gamelocs = reg_season.find_all(attrs = {'data-stat':'game_location'})
    locs = [entry.get_text() for entry in gamelocs if entry.get_text() == '' or entry.get_text() == '@']
    loc = list(map(lambda x: 1 if x =='' else 0, locs))

    result = [1 if (teampoints[i] - opppoints[i] > 0) else 0 for i in range(len(teampoints)) ]    

    links = reg_season.find_all(attrs = {'data-stat':'date_game'})
    urls = []
    for i in links:
        if i.find('a'):
            urls.append(i.find('a').attrs['href'])

    FG = []
    FGA = []
    threeP = []
    threePA = []
    FT = []
    FTA = []
    ORB = []
    TRB = []
    AST = []
    STL = []
    BLK = []
    TOV = []
    PF = []
    PTS = []

    for i, url in enumerate(urls):
        url = f'https://www.basketball-reference.com{url}'
        res = requests.get(url)
        res.raise_for_status()
        soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'lxml')
        opp = opp_abbrev[i].lower()
        gameteam = soup.find('div',{'id':f'div_box-score-{team.lower()}'})
        gameteam = pd.read_html(str(gameteam.find('table')), flavor='bs4')[0]
        gameteam = gameteam.dropna()

        FG.append(gameteam.iloc[-1]['FG'])
        FGA.append(gameteam.iloc[-1]['FGA'])
        threeP.append(gameteam.iloc[-1]['3P'])
        threePA.append(gameteam.iloc[-1]['3PA'])
        FT.append(gameteam.iloc[-1]['FT'])
        FTA.append(gameteam.iloc[-1]['FTA'])
        ORB.append(gameteam.iloc[-1]['ORB'])
        TRB.append(gameteam.iloc[-1]['TRB'])
        AST.append(gameteam.iloc[-1]['AST'])
        STL.append(gameteam.iloc[-1]['STL'])
        BLK.append(gameteam.iloc[-1]['BLK'])
        TOV.append(gameteam.iloc[-1]['TOV'])
        PF.append(gameteam.iloc[-1]['PF'])
        PTS.append(gameteam.iloc[-1]['PTS'])

    data = {"Team": [team] *len(dates), "Opp":opp_abbrev, "Game Num":list(range(1, len(dates)+1)), "Date":dates,
       "Location": loc, "FG":FG, 'FGA':FGA, '3P':threeP, '3PA':threePA, 'FT':FT,
       'FTA':FTA, 'ORB':ORB, 'TRB':TRB, 'AST':AST, 'STL':STL,'BLK':BLK,
       'TOV':TOV, 'PF':PF, 'PTS':PTS}

    db = pd.DataFrame(data)

    db['Date'] = pd.to_datetime(db['Date'])
    db['Date'] = db['Date'].apply(date_formatter)

    return db


In [101]:
def wnba_update(year):

    def line_scrape(dates):

        # Date format [YYYMMDD]

        bet_abbrevs = {
            'Atlanta Dream': 'ATL',
            'Washington Mystics': 'WAS',
            'Connecticut Sun': 'CON',
            'Chicago Sky': 'CHI',
            'New York Liberty': 'NYL',
            'Indiana Fever': 'IND',
            'Seattle Storm': 'SEA',
            'Phoenix Mercury': 'PHO',
            'L.A. Sparks':'LAS',
            'Minnesota Lynx': 'MIN',
            'Dallas Wings': 'DAL',
            'Las Vegas Aces': 'LVA'
        }

        scraped_dates = []
        moneydb = pd.DataFrame()

        for date in dates:

            if date not in scraped_dates:
                #print("Date: ", date)
                url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/money-line/?date={date}'
                res = requests.get(url, 'html.parser')
                res.raise_for_status()
                soup = bs4.BeautifulSoup(res.text, 'lxml')
                linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
                test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
                teams = [bet_abbrevs[team.get_text()] for team in test_teams]
                test_lines = linesoup.find_all('span', {'class':'opener'})
                lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
                #print("teams:",len(teams),"lines",len(lines))

                if len(teams) != len(lines):
                    pass
                else:
                    moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}

                    MLdb = pd.DataFrame.from_dict(moneylines)
                    moneydb = pd.concat([moneydb, MLdb])

                scraped_dates.append(date)
        print(moneydb.reset_index(drop = True))
        return moneydb.reset_index(drop = True)
        
    team_abbrevs18 = {
    'Atlanta Dream': 'ATL',
    'Washington Mystics': 'WAS',
    'Connecticut Sun': 'CON',
    'Chicago Sky': 'CHI',
    'New York Liberty': 'NYL',
    'Indiana Fever': 'IND',
    'Seattle Storm': 'SEA',
    'Phoenix Mercury': 'PHO',
    'Los Angeles Sparks':'LAS',
    'Minnesota Lynx': 'MIN',
    'Dallas Wings': 'DAL',
    'Las Vegas Aces': 'LVA'
    }
    
    
    db = pd.DataFrame()
    
    for team in list(team_abbrevs18.values()):
        print(team)
        db_running = wnba_scrape(team,year)
        db = pd.concat([db, db_running])
    
    db = db.reset_index(drop = True)
    
    moneydb = line_scrape(list(db['Date'].unique()))
    
    db['Odds'] = np.zeros(len(db['Team']))
    
    for i, date in enumerate(db['Date']):
    
        if date in list(moneydb['Date']):

            db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
            #print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])

        else:
            db['Odds'][i] = np.nan
    
    return db
    

In [64]:
test= wnba_update(2019)

ATL
WAS
CON
CHI
NYL
IND
SEA
PHO
LAS
MIN
DAL
LVA
   Team ML Odds      Date
0   DAL    +315  20190524
1   ATL    -380  20190524
2   IND    +115  20190524
3   NYL    -135  20190524
4   SEA    +245  20190531
5   ATL    -265  20190531
6   LVA    -118  20190531
7   PHO    -102  20190531
8   CON    +115  20190531
9   LAS    -125  20190531
10  NYL    +140  20190601
11  IND    -160  20190601
12  ATL    -375  20190601
13  WAS    +285  20190601
14  SEA    -103  20190601
15  CHI    -117  20190601
16  MIN    -135  20190601
17  DAL    +115  20190601
18  PHO    -155  20190525
19  SEA    +135  20190525
20  WAS    +100  20190525
21  CON    -120  20190525
22  CHI    +155  20190525
23  MIN    -175  20190525
24  IND    +425  20190528
25  CON    -550  20190528
26  CON    +215  20190602
27  LVA    -235  20190602
28  SEA    +112  20190529
29  MIN    -132  20190529
30  LAS    +135  20190526
31  LVA    -155  20190526


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [65]:
test

,Team,Opp,Game Num,Date,Location,FG,FGA,3P,3PA,FT,FTA,ORB,TRB,AST,STL,BLK,TOV,PF,PTS,Odds
0,ATL,DAL,1,20190524,1,27.0,63.0,4.0,14.0,18.0,29.0,8.0,31.0,16.0,6.0,11.0,15.0,13.0,76.0,-380.0
1,ATL,SEA,2,20190531,1,24.0,73.0,6.0,23.0,12.0,14.0,9.0,40.0,15.0,4.0,8.0,14.0,18.0,66.0,-265.0
2,ATL,WAS,3,20190601,0,30.0,71.0,5.0,21.0,10.0,17.0,12.0,28.0,9.0,5.0,1.0,13.0,19.0,75.0,-375.0
3,WAS,CON,1,20190525,0,27.0,72.0,3.0,21.0,12.0,15.0,11.0,37.0,17.0,4.0,7.0,11.0,17.0,69.0,100.0
4,WAS,ATL,2,20190601,1,38.0,67.0,9.0,25.0,11.0,13.0,9.0,35.0,27.0,6.0,6.0,14.0,22.0,96.0,285.0
5,CON,WAS,1,20190525,1,33.0,76.0,6.0,20.0,12.0,14.0,11.0,40.0,18.0,7.0,5.0,9.0,19.0,84.0,-120.0
6,CON,IND,2,20190528,1,32.0,70.0,10.0,22.0,14.0,22.0,12.0,37.0,18.0,4.0,4.0,12.0,11.0,88.0,-550.0
7,CON,LAS,3,20190531,0,24.0,78.0,6.0,23.0,16.0,25.0,18.0,51.0,13.0,4.0,5.0,14.0,21.0,70.0,115.0
8,CON,LVA,4,20190602,0,26.0,77.0,8.0,26.0,20.0,23.0,11.0,39.0,17.0,14.0,8.0,9.0,19.0,80.0,215.0
9,CHI,MIN,1,20190525,0,28.0,70.0,3.0,13.0,12.0,15.0,6.0,29.0,19.0,11.0,3.0,15.0,22.0,71.0,155.0


In [70]:
def db_format(db):
    
    def rolling_avgs(db):
        TSP4DB = pd.DataFrame()
        rollingTSP = []
        for team in db['Team'].unique():
            teamRollingTSP = db[db['Team'] == team]['TS%'].rolling(4).mean()
            rollingTSP.append(teamRollingTSP)
            TSP = {"Rolling4TSP":teamRollingTSP}
            TSP4df = pd.DataFrame.from_dict(TSP)
            TSP4DB = pd.concat([TSP4DB, TSP4df])

        TOV4DB = pd.DataFrame()
        rollingTOV = []
        for team in db['Team'].unique():
            teamRollingTOV = db[db['Team'] == team]['TOV%'].rolling(4).mean()
            rollingTOV.append(teamRollingTOV)
            TOV = {"Rolling4TOV":teamRollingTOV}
            TOV4df = pd.DataFrame.from_dict(TOV)
            TOV4DB = pd.concat([TOV4DB, TOV4df])

        ORB4DB = pd.DataFrame()
        rollingORB = []
        for team in db['Team'].unique():
            teamRollingORB = db[db['Team'] == team]['ORB%'].rolling(4).mean()
            rollingORB.append(teamRollingORB)
            ORB = {"Rolling4ORB":teamRollingORB}
            ORB4df = pd.DataFrame.from_dict(ORB)
            ORB4DB = pd.concat([ORB4DB, ORB4df])

        FTR4DB = pd.DataFrame()
        rollingFTR = []
        for team in db['Team'].unique():
            teamRollingFTR = db[db['Team'] == team]['FTR'].rolling(4).mean()
            rollingFTR.append(teamRollingFTR)
            FTR = {"Rolling4FTR":teamRollingFTR}
            FTR4df = pd.DataFrame.from_dict(FTR)
            FTR4DB = pd.concat([FTR4DB, FTR4df])

        Poss4DB = pd.DataFrame()
        rollingPoss = []
        for team in db['Team'].unique():
            teamRollingPoss = db[db['Team'] == team]['Poss'].rolling(4).mean()
            rollingPoss.append(teamRollingPoss)
            POSS = {"Rolling4Poss":teamRollingPoss}
            PACE4df = pd.DataFrame.from_dict(POSS)
            Poss4DB = pd.concat([Poss4DB, PACE4df])

        OEff4DB = pd.DataFrame()
        rollingOEff = []
        for team in db['Team'].unique():
            teamRollingOEff = db[db['Team'] == team]['OEff'].rolling(4).mean()
            rollingOEff.append(teamRollingOEff)
            OEff = {"Rolling4OEff":teamRollingOEff}
            OEff4df = pd.DataFrame.from_dict(OEff)
            OEff4DB = pd.concat([OEff4DB, OEff4df])

        DEff4DB = pd.DataFrame()
        rollingDEff = []
        for team in db['Team'].unique():
            teamRollingDEff = db[db['Team'] == team]['DEff'].rolling(4).mean()
            rollingDEff.append(teamRollingDEff)
            DEff = {"Rolling4DEff":teamRollingDEff}
            DEff4df = pd.DataFrame.from_dict(DEff)
            DEff4DB = pd.concat([DEff4DB, DEff4df])

        db = pd.concat([db, TSP4DB, TOV4DB, ORB4DB, FTR4DB, Poss4DB, OEff4DB, DEff4DB], axis =1)

        return db

    db['Opp PTS'] = np.zeros(len(db))
    db['Opp TRB'] = np.zeros(len(db))
    db['Opp ORB'] = np.zeros(len(db))

    for i, opp in enumerate(db['Opp']):
        db['Opp PTS'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['PTS']
        db['Opp TRB'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['TRB']
        db['Opp ORB'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['ORB']        

    db['TS%'] = np.zeros(len(db))
    db['TOV%'] = np.zeros(len(db))
    db['ORB%'] = np.zeros(len(db))
    db['FTR'] = np.zeros(len(db))
    db['Poss'] = np.zeros(len(db))
    db['OEff'] = np.zeros(len(db))
    db['DEff'] = np.zeros(len(db))

    for i in range(len(db)):
        db['TS%'][i] = db['PTS'][i] / (db['FGA'][i] + .44*db['FTA'][i]) * 0.5
        db['TOV%'][i] = db['TOV'][i] / (db['FGA'][i] + .44*db['FTA'][i] + db['TOV'][i])
        db['ORB%'][i] = db['ORB'][i] / (db['ORB'][i] + (db['Opp TRB'][i] - db['Opp ORB'][i]))
        db['FTR'][i] = db['FT'][i] / db['FGA'][i]
        db['Poss'][i] = 0.96 * db['FGA'][i] + db['TOV'][i] + 0.44*db['FTA'][i] - db['ORB'][i]
        db['OEff'][i] = 100* db['PTS'][i]/db['Poss'][i]
        # db['DEff'][i] = opponenets OEff; use indexing after this step

    db['Opp TS%'] = np.zeros(len(db))
    db['Opp TOV%'] = np.zeros(len(db))
    db['Opp ORB%'] = np.zeros(len(db))
    db['Opp FTR'] = np.zeros(len(db))
    db['Opp Poss'] = np.zeros(len(db))
    db['Opp OEff'] = np.zeros(len(db))
    db['Opp DEff'] = np.zeros(len(db))

    for i, opp in enumerate(db['Opp']):
        db['DEff'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['OEff']
        db['Opp TS%'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['TS%']
        db['Opp TOV%'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['TOV%']
        db['Opp ORB%'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['ORB%']
        db['Opp FTR'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['FTR']
        db['Opp Poss'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Poss']
        db['Opp OEff'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['OEff']

    for i, opp in enumerate(db['Opp']):
        db['Opp DEff'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['DEff']

    db = rolling_avgs(db)
    
    db['Opp R4TSP'] = np.zeros(len(db['Team']))
    db['Opp R4TOV'] = np.zeros(len(db['Team']))
    db['Opp R4ORB'] = np.zeros(len(db['Team']))
    db['Opp R4FTR'] = np.zeros(len(db['Team']))
    db['Opp R4Poss'] = np.zeros(len(db['Team']))
    db['Opp R4OEff'] = np.zeros(len(db['Team']))
    db['Opp R4DEff'] = np.zeros(len(db['Team']))

    for i, opp in enumerate(db['Opp']):
        db['Opp R4TSP'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4TSP']
        db['Opp R4TOV'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4TOV']
        db['Opp R4ORB'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4ORB']
        db['Opp R4FTR'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4FTR']
        db['Opp R4Poss'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4Poss']
        db['Opp R4OEff'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4OEff']
        db['Opp R4DEff'][i] = db[(db['Team'] == opp) & (db['Date'] == db['Date'][i])]['Rolling4DEff']
    
    
    return db


In [71]:
test1 = db_format(test)

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\

In [72]:
test1

,Team,Opp,Game Num,Date,Location,FG,FGA,3P,3PA,FT,...,Rolling4Poss,Rolling4OEff,Rolling4DEff,Opp R4TSP,Opp R4TOV,Opp R4ORB,Opp R4FTR,Opp R4Poss,Opp R4OEff,Opp R4DEff
0,ATL,DAL,1,20190524,1,27.0,63.0,4.0,14.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ATL,SEA,2,20190531,1,24.0,73.0,6.0,23.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ATL,WAS,3,20190601,0,30.0,71.0,5.0,21.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,WAS,CON,1,20190525,0,27.0,72.0,3.0,21.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WAS,ATL,2,20190601,1,38.0,67.0,9.0,25.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,CON,WAS,1,20190525,1,33.0,76.0,6.0,20.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,CON,IND,2,20190528,1,32.0,70.0,10.0,22.0,14.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,CON,LAS,3,20190531,0,24.0,78.0,6.0,23.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,CON,LVA,4,20190602,0,26.0,77.0,8.0,26.0,20.0,...,79.48,101.597408,94.474576,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,CHI,MIN,1,20190525,0,28.0,70.0,3.0,13.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
def today_games(date):
    
    # Date format YYYMMDD
    
    bet_abbrevs = {
        'Atlanta Dream': 'ATL',
        'Washington Mystics': 'WAS',
        'Connecticut Sun': 'CON',
        'Chicago Sky': 'CHI',
        'New York Liberty': 'NYL',
        'Indiana Fever': 'IND',
        'Seattle Storm': 'SEA',
        'Phoenix Mercury': 'PHO',
        'L.A. Sparks':'LAS',
        'Minnesota Lynx': 'MIN',
        'Dallas Wings': 'DAL',
        'Las Vegas Aces': 'LVA'
    }

    moneydb = pd.DataFrame()
    
    print("Date: ", date)
    url = f'https://www.sportsbookreview.com/betting-odds/wnba-basketball/money-line/?date={date}'
    res = requests.get(url, 'html.parser')
    res.raise_for_status()
    soup = bs4.BeautifulSoup(res.text, 'lxml')
    linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
    test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
    teams = [bet_abbrevs[team.get_text()] for team in test_teams]
    test_lines = linesoup.find_all('span', {'class':'opener'})
    lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
    #print("teams:",len(teams),"lines",len(lines))

    while len(teams) != len(lines):
        lines.append(np.nan)
        
    moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}

    MLdb = pd.DataFrame.from_dict(moneylines)
    moneydb = pd.concat([moneydb, MLdb])
    
    if len(moneydb) == 0:
        print ("No games today")
    else:
        return moneydb.reset_index(drop = True)

In [40]:
games = today_games(20190531)

Date:  20190531


In [41]:
games

,Team,ML Odds,Date
0,SEA,+245,20190531
1,ATL,-265,20190531
2,LVA,-118,20190531
3,PHO,-102,20190531
4,CON,+115,20190531
5,LAS,-125,20190531


In [104]:
def predict_games(db, games):
    db14 = pd.read_csv('WNBA14.csv')
    db15 = pd.read_csv('WNBA15.csv')
    db16 = pd.read_csv('WNBA16.csv')
    db17 = pd.read_csv('WNBA17.csv')
    db18 = pd.read_csv('WNBA18.csv')
    
    predict_db = pd.concat([db14, db15, db16, db17, db18]).reset_index(drop = True)
    print(predict_db.info())
    Team = []
    Location = []
    Rolling4TSP = []
    Rolling4TOV = []
    Rolling4ORB = []
    Rolling4FTR = []
    Rolling4Poss = []
    OppR4Poss = []
    OppR4TSP = []
    OppR4TOV = []
    OppR4ORB = []
    OppR4FTR = []
    Rolling4DEff = []
    OppR4DEff = []
    Rolling4OEff = []
    OppR4OEff = []
    
    for i, team in enumerate(games['Team']):
        if i%2 == 0:
            Location.append(0)
        else:
            Location.append(1)
        
        Team.append(team)
        Rolling4TSP.append(db[db['Team'] == team]['Rolling4TSP'].iloc[-1])
        Rolling4TOV.append(db[db['Team'] == team]['Rolling4TOV'].iloc[-1])
        Rolling4ORB.append(db[db['Team'] == team]['Rolling4ORB'].iloc[-1])
        Rolling4FTR.append(db[db['Team'] == team]['Rolling4FTR'].iloc[-1])
        Rolling4Poss.append(db[db['Team'] == team]['Rolling4Poss'].iloc[-1])
        OppR4Poss.append(db[db['Team'] == team]['Opp R4Poss'].iloc[-1])
        OppR4TSP.append(db[db['Team'] == team]['Opp R4TSP'].iloc[-1])
        OppR4TOV.append(db[db['Team'] == team]['Opp R4TOV'].iloc[-1])
        OppR4ORB.append(db[db['Team'] == team]['Opp R4ORB'].iloc[-1])
        OppR4FTR.append(db[db['Team'] == team]['Opp R4FTR'].iloc[-1])
        Rolling4DEff.append(db[db['Team'] == team]['Rolling4DEff'].iloc[-1])
        OppR4DEff.append(db[db['Team'] == team]['Opp R4DEff'].iloc[-1])
        Rolling4OEff.append(db[db['Team'] == team]['Rolling4OEff'].iloc[-1])
        OppR4OEff.append(db[db['Team'] == team]['Opp R4OEff'].iloc[-1])
    
    data = {"Team": Team, "Location":Location, 'Rolling4TSP': Rolling4TSP, 'Rolling4TOV':Rolling4TOV, 'Rolling4ORB':Rolling4ORB,
           "Rolling4FTR":Rolling4FTR, 'Rolling4Poss':Rolling4Poss, 'Opp R4Poss':OppR4Poss, 'Opp R4TSP':OppR4TSP, 'Opp R4TOV':OppR4TOV,
           "Opp R4ORB":OppR4ORB, 'Opp R4FTR':OppR4FTR, 'Rolling4DEff':Rolling4DEff, 'Opp R4DEff':OppR4DEff, 
           'Rolling4OEff':Rolling4OEff, 'Opp R4OEff':OppR4OEff}
    
    data = pd.DataFrame(data)
    
    featureCols = ['Location','Rolling4TSP','Rolling4TOV','Rolling4ORB',
                   'Rolling4FTR','Rolling4Poss','Opp R4Poss','Opp R4TSP',
                   'Opp R4TOV','Opp R4ORB','Opp R4FTR', 'Rolling4DEff', 
                   'Opp R4DEff', 'Rolling4OEff','Opp R4OEff']
    target = ['Result']

    cols =  ['Result', 'Location','Rolling4TSP','Rolling4TOV','Rolling4ORB','Rolling4FTR','Rolling4Poss','Opp R4Poss','Opp R4TSP','Opp R4TOV','Opp R4ORB','Opp R4FTR', 'Rolling4DEff', 'Opp R4DEff', 'Rolling4OEff','Opp R4OEff']
    predict_db = predict_db[cols].dropna()
    
    X = predict_db[featureCols]
    y = np.array(predict_db[target]).flatten()

    preds_avg = []

    for i in range(20):
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        ada = AdaBoostClassifier(n_estimators=100, learning_rate=0.1).fit(x_train, y_train)

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        gbc = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, max_depth=1).fit(x_train, y_train)

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
        eclf1 = VotingClassifier(estimators=[('ada', ada), ('gbc', gbc)], voting='soft')
        eclf1 = eclf1.fit(x_train, y_train)

        pred = eclf1.predict(data[featureCols])

        preds_avg.append(pred)
        
    preds = [sum(i) for i in zip(*preds_avg)]

    return data, preds
    

In [105]:
predict_games(test1, games)

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 39 columns):
0               1196 non-null float64
DEff            2038 non-null float64
Date            2038 non-null object
FTR             2038 non-null float64
Game Num        2038 non-null int64
Location        2038 non-null int64
OEff            2038 non-null float64
ORB             2038 non-null float64
Odds            1196 non-null float64
Opp             2038 non-null object
Opp DEff        2038 non-null float64
Opp FTR         2038 non-null float64
Opp OEff        2038 non-null float64
Opp ORB         2038 non-null float64
Opp Poss        2038 non-null float64
Opp R4DEff      1798 non-null float64
Opp R4FTR       1798 non-null float64
Opp R4OEff      1798 non-null float64
Opp R4ORB       1798 non-null float64
Opp R4Poss      1798 non-null float64
Opp R4TOV       1798 non-null float64
Opp R4TSP       1798 non-null float64
Opp Score       2038 non-null float64
Opp TOV         2038 non-

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [132]:
test1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 51 columns):
Team            16 non-null object
Opp             16 non-null object
Game Num        16 non-null int64
Date            16 non-null object
Location        16 non-null int64
FG              16 non-null float64
FGA             16 non-null float64
3P              16 non-null float64
3PA             16 non-null float64
FT              16 non-null float64
FTA             16 non-null float64
ORB             16 non-null float64
TRB             16 non-null float64
AST             16 non-null float64
STL             16 non-null float64
BLK             16 non-null float64
TOV             16 non-null float64
PF              16 non-null float64
PTS             16 non-null float64
Odds            16 non-null float64
Opp PTS         16 non-null float64
Opp TRB         16 non-null float64
Opp ORB         16 non-null float64
TS%             16 non-null float64
TOV%            16 non-null float64
ORB%